USER REVIEWS

In [274]:
# se importan las librerias que se van a usar 
import gzip
import ast
import chardet
import pandas as pd
import nltk
import seaborn as sns

Se accede a la informacion del archivo comprimido

In [275]:
file_path_user_reviews = '../datasets/user_reviews.json.gz'
lista=[]
with gzip.open(file_path_user_reviews, 'rt', encoding='MacRoman') as archivo:
    for line in archivo.readlines():
        lista.append(ast.literal_eval(line))

user_review = pd.DataFrame(lista)
user_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


se procede a desanidar la columna reviews que se necesitara mas adelante pare el proceso 

In [276]:
explode_df_user_review = user_review.explode('reviews') # Duplico las filas generando un diccionario 

In [277]:
# Ahora se concateno el dataframe original, con el dataframe generado 
explode_df_user_review = pd.concat([explode_df_user_review.drop(['reviews'],axis=1),
                                    explode_df_user_review['reviews'].apply(pd.Series)],axis=1)  

In [278]:
explode_df_user_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      59333 non-null  object 
 1   user_url     59333 non-null  object 
 2   funny        59305 non-null  object 
 3   posted       59305 non-null  object 
 4   last_edited  59305 non-null  object 
 5   item_id      59305 non-null  object 
 6   helpful      59305 non-null  object 
 7   recommend    59305 non-null  object 
 8   review       59305 non-null  object 
 9   0            0 non-null      float64
dtypes: float64(1), object(9)
memory usage: 5.0+ MB


Se eliminan las columnas que no se van a usar 

In [279]:
explode_df_user_review.drop(columns=['user_url','funny','helpful','last_edited', 0],inplace=True)

Se realiza el analisis de sentimiento solicitado con NLP

Se va a crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1

In [280]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Descargar recursos de NLTK 
nltk.download('vader_lexicon')

# Crear una instancia de SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Crear una función para asignar el análisis de sentimiento
def assign_sentiment_analysis(review):
    if pd.isna(review):  # Si la reseña está ausente
        return 1  # Valor neutral
    else:
        # Realizar análisis de sentimiento con vaderSentiment
        compound_score = sia.polarity_scores(review)['compound']

        if compound_score >= 0.05:
            return 2  # Positivo
        elif compound_score <= -0.05:
            return 0  # Negativo
        else:
            return 1  # Neutral



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Willc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [281]:
explode_df_user_review['review'].fillna('',inplace=True) # Remplazo los nulos con un string vacio

In [282]:
# Aplicar la función a la columna 'review' y crear la nueva columna 'sentiment_analysis'
explode_df_user_review['sentiment_analysis'] = explode_df_user_review['review'].apply(assign_sentiment_analysis)

# Reemplazar la columna original 'review' con la nueva 'sentiment_analysis'
explode_df_user_review.drop(columns=['review'], inplace=True)

Verificamos valores nulos

In [283]:
explode_df_user_review.isnull().sum()

user_id                0
posted                28
item_id               28
recommend             28
sentiment_analysis     0
dtype: int64

In [284]:
explode_df_user_review[explode_df_user_review.isnull().any(axis=1)]

,user_id,posted,item_id,recommend,sentiment_analysis
62,gdxsd,NaN,NaN,NaN,1
83,76561198094224872,NaN,NaN,NaN,1
1047,76561198021575394,NaN,NaN,NaN,1
3954,cmuir37,NaN,NaN,NaN,1
5394,Jaysteeny,NaN,NaN,NaN,1
6135,ML8989,NaN,NaN,NaN,1
7583,76561198079215291,NaN,NaN,NaN,1
7952,76561198079342142,NaN,NaN,NaN,1
9894,76561198061996985,NaN,NaN,NaN,1
10381,76561198108286351,NaN,NaN,NaN,1


ya que las filas estan completamente vacias se procede a eliminarlas

In [285]:
explode_df_user_review.dropna(inplace=True)

In [286]:
explode_df_user_review.head()

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,"Posted November 5, 2011.",1250,True,2
0,76561197970982479,"Posted July 15, 2011.",22200,True,2
0,76561197970982479,"Posted April 21, 2011.",43110,True,2
1,js41637,"Posted June 24, 2014.",251610,True,2
1,js41637,"Posted September 8, 2013.",227300,True,2


verificamos los datos duplicados

In [287]:
explode_df_user_review[explode_df_user_review.duplicated()].count()

user_id               874
posted                874
item_id               874
recommend             874
sentiment_analysis    874
dtype: int64

Eliminamos las filas duplicadas ya que no nos aportan a nuestro analisis 

In [288]:
explode_df_user_review = explode_df_user_review.drop_duplicates()#eliminamos las filas duplicadas

se procede a eliminar las columnas que no son necesarias para el procedimiento

In [289]:
explode_df_user_review.shape

(58431, 5)

In [290]:
explode_df_user_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58431 entries, 0 to 25798
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             58431 non-null  object
 1   posted              58431 non-null  object
 2   item_id             58431 non-null  object
 3   recommend           58431 non-null  object
 4   sentiment_analysis  58431 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.7+ MB


Se identifica que para la funcion 3 se debe crear la columna year con los años de posted.

In [295]:
explode_df_user_review['year'] = explode_df_user_review['posted'].str.extract(r'(\d{4})')#estrae de la columna posted 4 digitos seguidos que corresponden al año

In [298]:
explode_df_user_review.isnull().sum()#verificamos los valores nulos

user_id                  0
posted                   0
item_id                  0
recommend                0
sentiment_analysis       0
year                  9933
dtype: int64

ya que estas filas nulas en la columna year contiene casi toda la informacion menos el año, no se eliminaran pero se fitraran para ciertas funciones en las que se use la columna year

Por ultimo despuesde de haber realizado las transformaciones solicitadas y las que se consideraron necesarias se procede a exportar a un csv

In [299]:
explode_df_user_review.to_csv('../datasets/user_review.csv',index=False)